<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/makine_e%C4%9Fitim_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, recall_score, cohen_kappa_score, precision_score, accuracy_score

# --- 1. ADIM: VERİ ÜRETİMİ (TEKNOFEST Üniversite Şartnamesi) ---
n_samples = 200000
print(f"TEKNOFEST 2026 Üniversite Şartnamesine uygun {n_samples} veri üretiliyor...")

def generate_university_level_data(n):
    dna_seqs = []
    protein_seqs = []
    bio_features = []
    labels = []
    amino_acids = list("ACDEFGHIKLMNPQRSTVWY")

    for _ in range(n):
        # Genomik ve Proteomik Komşuluk (11 birim)
        dna_fragment = ''.join(np.random.choice(list('ACGT'), size=11))
        prot_fragment = ''.join(np.random.choice(amino_acids, size=11))

        # Biyolojik Özellikler
        risk_score = np.random.beta(2, 2)
        maf = np.random.exponential(0.05)
        if maf > 0.5: maf = 0.5
        conservation = np.random.uniform(0, 10)
        hydrophobicity_change = np.random.uniform(-5, 5)
        polarity_change = np.random.uniform(-3, 3)
        mol_weight_change = np.random.uniform(-50, 50)

        # Etiketleme Mantığı
        pathogenicity_index = (risk_score * 0.4) + (conservation/10 * 0.2) + ((0.5 - maf)*2 * 0.2) + (abs(hydrophobicity_change)/5 * 0.2)
        pathogenicity_index += np.random.normal(0, 0.05)

        if pathogenicity_index > 0.65:
            labels.append(1) # Patojenik
        else:
            labels.append(0) # Benign

        dna_seqs.append(dna_fragment)
        protein_seqs.append(prot_fragment)
        bio_features.append([risk_score, maf, conservation, hydrophobicity_change, polarity_change, mol_weight_change])

    return dna_seqs, protein_seqs, np.array(bio_features), np.array(labels)

dna_sequences, protein_sequences, X_numerical, y = generate_university_level_data(n_samples)

# --- 2. ADIM: VERİ KAYDI ---
print(">> Veriler 'teknofest_univ_egitim_seti.txt' dosyasına kaydediliyor...")
df_data = pd.DataFrame(X_numerical, columns=['Risk_Skoru', 'MAF', 'Korunmusluk', 'Hidrofobiklik', 'Polarite', 'Mol_Agirlik'])
df_data.insert(0, 'Genomik_Komsuluk', dna_sequences)
df_data.insert(1, 'Proteomik_Komsuluk', protein_sequences)
df_data['SINIF_ETIKETI'] = y
df_data.to_csv('teknofest_univ_egitim_seti.txt', sep='\t', index=False)
print(">> Kayıt Başarılı!\n")

# --- 3. ADIM: ÖN İŞLEME ---
dna_tokenizer = Tokenizer(char_level=True)
dna_tokenizer.fit_on_texts(dna_sequences)
X_dna = pad_sequences(dna_tokenizer.texts_to_sequences(dna_sequences), maxlen=11, padding='post')

prot_tokenizer = Tokenizer(char_level=True)
prot_tokenizer.fit_on_texts(protein_sequences)
X_prot = pad_sequences(prot_tokenizer.texts_to_sequences(protein_sequences), maxlen=11, padding='post')

X_num = X_numerical

X_dna_train, X_dna_test, X_prot_train, X_prot_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_dna, X_prot, X_num, y, test_size=0.2, random_state=42
)

# --- 4. ADIM: MODEL MİMARİSİ ---
input_dna = Input(shape=(11,), name="DNA_Input")
emb_dna = Embedding(len(dna_tokenizer.word_index)+1, 8, input_length=11)(input_dna)
pool_dna = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_dna))

input_prot = Input(shape=(11,), name="Protein_Input")
emb_prot = Embedding(len(prot_tokenizer.word_index)+1, 8, input_length=11)(input_prot)
pool_prot = GlobalMaxPooling1D()(Conv1D(16, 3, activation='relu')(emb_prot))

input_num = Input(shape=(6,), name="Bio_Features_Input")
norm_num = BatchNormalization()(Dense(16, activation='relu')(input_num))

merged = Concatenate()([pool_dna, pool_prot, norm_num])
z = Dropout(0.4)(Dense(64, activation='relu')(merged))
z = Dense(32, activation='relu')(z)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[input_dna, input_prot, input_num], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 5. ADIM: EĞİTİM ---
print("Model Eğitiliyor...")
model.fit([X_dna_train, X_prot_train, X_num_train], y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

# --- 6. ADIM: DETAYLI PERFORMANS ANALİZİ ---
print("\n" + "="*50)
print("   TEKNOFEST MODEL PERFORMANS RAPORU")
print("="*50)

y_pred_prob = model.predict([X_dna_test, X_prot_test, X_num_test])
y_pred = (y_pred_prob > 0.5).astype(int)

# Metriklerin Hesaplanması
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

print(f"✅ Accuracy (Doğruluk)     : {acc:.4f}")
print(f"🎯 Recall (Duyarlılık)     : {rec:.4f}  <-- Hastaları yakalama oranı")
print(f"👌 Precision (Kesinlik)    : {prec:.4f}")
print(f"🏆 F1 Score (Denge Skoru)  : {f1:.4f}  <-- Şartname Kriteri")
print(f"⚖️  Cohen's Kappa           : {kappa:.4f}  <-- Şans faktöründen arındırılmış başarı")

print("-" * 50)
print("DETAYLI SINIFLANDIRMA RAPORU:")
print(classification_report(y_test, y_pred, target_names=['Benign (Sağlıklı)', 'Patojenik (Hasta)']))
print("="*50)

# --- 7. ADIM: SONUÇLARI KAYDETME ---
test_dna_texts = [t.replace(' ', '').upper() for t in dna_tokenizer.sequences_to_texts(X_dna_test)]
test_prot_texts = [t.replace(' ', '').upper() for t in prot_tokenizer.sequences_to_texts(X_prot_test)]

df_result = pd.DataFrame(X_num_test, columns=['Risk_Skoru', 'MAF', 'Korunmusluk', 'Hidrofobiklik', 'Polarite', 'Mol_Agirlik'])
df_result.insert(0, 'DNA_Dizisi', test_dna_texts)
df_result.insert(1, 'Protein_Dizisi', test_prot_texts)
df_result['Gercek_Deger'] = y_test
df_result['Tahmin_Degeri'] = y_pred.flatten()
df_result['Olasilik'] = y_pred_prob.flatten()
df_result['SONUC'] = np.where(df_result['Gercek_Deger'] == df_result['Tahmin_Degeri'], 'DOGRU', 'YANLIS')

df_result.to_csv('teknofest_univ_test_sonuclari.txt', sep='\t', index=False)
print(">> Detaylı sonuçlar 'teknofest_univ_test_sonuclari.txt' dosyasına kaydedildi.")

TEKNOFEST 2026 Üniversite Şartnamesine uygun 200000 veri üretiliyor...
>> Veriler 'teknofest_univ_egitim_seti.txt' dosyasına kaydediliyor...
>> Kayıt Başarılı!

Model Eğitiliyor...
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2250/2250 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.7764 - loss: 0.4536 - val_accuracy: 0.8914 - val_loss: 0.2410
Epoch 2/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8829 - loss: 0.2638 - val_accuracy: 0.8873 - val_loss: 0.2461
Epoch 3/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8843 - loss: 0.2583 - val_accuracy: 0.8905 - val_loss: 0.2397
Epoch 4/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8857 - loss: 0.2537 - val_accuracy: 0.8871 - val_loss: 0.2429
Epoch 5/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8845 - loss: 0.2577 - val_accuracy: 0.8898 - val_loss: 0.2422
Epoch 6/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8838 - loss: 0.2562 - val_accuracy: 0.8882 - val_loss: 0.2432
Epoch 7/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8850 - loss: 0.2554 - val_accuracy: 0.8904 - val_loss: 0.2414
Epoch 8/10
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8874 - loss: 0.2537 - val_a